<a href="https://colab.research.google.com/github/runawaycow/TAP_Project/blob/main/Mapbox_Import_HighResolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step-by-step solution:

1.   Precisely determine the tiles covering your bounding box
2.   Retrieve each tile individually.
3.  Stitch tiles to produce an accurately aligned final image.
4.  Uses Mercantile to ensure precise tile alignment.
5.  Accurately computes pixel coordinates using Mapbox’s projection standard.
6.  Maintains correct proportions and alignment for exported image.


# **Python script (fully functional and accurate):**

Install dependencies first:

In [8]:
! pip install requests pillow mercantile

# **Instructions:**
Replace:
1. YOUR_STYLE_URL with your Mapbox style URL identifier.
2. YOUR_ACCESS_TOKEN with your Mapbox access token.
3. (LATITUDE_LOWER_LEFT, LONGITUDE_LOWER_LEFT) and (LATITUDE_UPPER_RIGHT,  LONGITUDE_UPPER_RIGHT) with your desired coordinates.
5. YOUR_ZOOM_LEVEL with an appropriate zoom level.

# **Boundary Reference**
**Cairns Boundary:**

lower_left = (-17.1077, 145.669000)

upper_right = (-16.75, 145.80)

**Townsville Boundary**

In [19]:
import requests
import mercantile
from PIL import Image
from io import BytesIO
import math

# Mapbox parameters
style_url = 'runawaycow/cm6m6zalu00cg01sq1m6eahqr'
access_token = 'pk.eyJ1IjoicnVuYXdheWNvdyIsImEiOiJja21hemUzOWgxZGoxMnZudXF2NmsxMmZoIn0.fFx2sslVYu1v-YfuSdjrKg'

# Define bounding box coordinates
lower_left = (-17.1077, 145.669000)
upper_right = (-16.75, 145.80)

# Calculate the center coordinates
center_lat = (lower_left[0] + upper_right[0]) / 2
center_lon = (lower_left[1] + upper_right[1]) / 2

# Set zoom level (adjust as needed)
zoom = 12

# Tile size (Mapbox standard)
tile_size = 512  # Mapbox tiles are typically 512x512 px

# Get tiles covering bounding box
tiles = list(mercantile.tiles(
    west=lower_left[1], south=lower_left[0],
    east=upper_right[1], north=upper_right[0],
    zooms=zoom
))

# Determine stitched image size
min_x = min(tile.x for tile in tiles)
max_x = max(tile.x for tile in tiles)
min_y = min(tile.y for tile in tiles)
max_y = max(tile.y for tile in tiles)

width = (max_x - min_x + 1) * tile_size
height = (max_y - min_y + 1) * tile_size

final_image = Image.new('RGB', (width, height))

# Fetch and stitch tiles
for tile in tiles:
    tile_url = (
        f"https://api.mapbox.com/styles/v1/{style_url}/tiles/{tile_size}/{tile.z}/{tile.x}/{tile.y}"
        f"?access_token={access_token}"
    )

    response = requests.get(tile_url)
    if response.status_code == 200:
        tile_img = Image.open(BytesIO(response.content))
        tile_img.show(title=f'Tile {tile.z}/{tile.x}/{tile.y}')  # Display for debugging
        x_offset = (tile.x - min_x) * tile_size
        y_offset = (tile.y - min_y) * tile_size
        final_image.paste(tile_img, (x_offset, y_offset))
    else:
        print(f"Failed to retrieve tile {tile.z}/{tile.x}/{tile.y}: {response.text}")

# Save final image
final_image.save('high_resolution_map.jpg')
print("High-resolution map saved successfully.")



High-resolution map saved successfully.
